In [1]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 3.1/72.0 MB 15.3 MB/s eta 0:00:05
   ---- ----------------------------------- 7.9/72.0 MB 20.3 MB/s eta 0:00:04
   ---------- ----------------------------- 18.1/72.0 MB 29.2 MB/s eta 0:00:02
   ---------------- ----------------------- 30.4/72.0 MB 37.1 MB/s eta 0:00:02
   ------------------------ --------------- 44.6/72.0 MB 43.6 MB/s eta 0:00:01
   -------------------------------- ------- 59.2/72.0 MB 48.4 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 51.4 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 48.3 MB/s  0:00:01

   ------------- -------------------------- 1/3 [xgboost]
   ------------- -------------------------- 1/3 [xgboost]
   ------------- -------------------------- 1/3 [xgboost]
   ------------- -------------------------- 1/3 [xgboost]
   ------------- -------------------------- 1/3 [xgboost]
   ---------

In [ ]:
!aws configure

In [2]:
import mlflow
# Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-52-53-234-96.us-west-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/10/24 21:41:55 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-tutorial-bucket/5', creation_time=1761367316164, experiment_id='5', last_update_time=1761367316164, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [5]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [7]:
# Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2025-10-24 21:52:09,545] A new study created in memory with name: no-name-e944dd88-3939-4380-bbf0-91a03fb0f11c
[I 2025-10-24 21:52:41,455] Trial 0 finished with value: 0.7335333424246556 and parameters: {'n_estimators': 125, 'learning_rate': 0.08630135686592105, 'max_depth': 5}. Best is trial 0 with value: 0.7335333424246556.
[I 2025-10-24 21:53:46,285] Trial 1 finished with value: 0.5460248193099686 and parameters: {'n_estimators': 209, 'learning_rate': 0.00023580981888612077, 'max_depth': 5}. Best is trial 0 with value: 0.7335333424246556.
[I 2025-10-24 21:54:24,422] Trial 2 finished with value: 0.7309423155597982 and parameters: {'n_estimators': 248, 'learning_rate': 0.07392732369486918, 'max_depth': 3}. Best is trial 0 with value: 0.7335333424246556.
[I 2025-10-24 21:55:12,199] Trial 3 finished with value: 0.5450702304650211 and parameters: {'n_estimators': 150, 'learning_rate': 0.0002466272183157816, 'max_depth': 5}. Best is trial 0 with value: 0.7335333424246556.
[I 2025-10-24

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-52-53-234-96.us-west-1.compute.amazonaws.com:5000/#/experiments/5/runs/987b6b59eff34cf4bd383fa7fe9c15c1
🧪 View experiment at: http://ec2-52-53-234-96.us-west-1.compute.amazonaws.com:5000/#/experiments/5
